In [47]:
import requests
import csv
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.util import ngrams
import lxml
from collections import defaultdict
import pandas as pd
import numpy as np
import os
from urllib.request import urlopen
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn import model_selection, preprocessing,svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import decomposition, ensemble
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
import string
import luigi
from dropbox.files import WriteMode
import dropbox
dbx = dropbox.Dropbox('aDf1htlkmN8AAAAAAAAC-_IjjyFsEJBJU6fY86Y32dr1LH3kVthriApkX9h08kmf')
os.remove(os.getcwd()+"/wordcount_dataset.csv")
os.remove(os.getcwd()+"/model_input.csv")
os.remove(os.getcwd()+"/Vader_output.csv")
os.remove(os.getcwd()+"/merged_data.csv")
class Task0(luigi.Task):
    def run(self):
        
        dbx.files_download_to_file(os.getcwd()+'/Final_dataset.csv', '/DataScienceTeam9/'+ 'Final_dataset.csv')

    def output(self):
        return luigi.LocalTarget(os.getcwd()+"/Final_dataset.csv")
class Task1(luigi.Task):
    def requires(self):
        yield Task0()
    def run(self):
        dbx.files_download_to_file(os.getcwd()+'/word_final.csv', '/DataScienceTeam9/'+ 'word_final.csv')

    def output(self):
        return luigi.LocalTarget(os.getcwd()+"/word_final.csv")
class Task2(luigi.Task):
    def requires(self):
        yield Task1()
    def run(self):
        def generate_ngrams(s, n):
            # Convert to lowercases
            s = s.lower()
    
            # Replace all none alphanumeric characters with spaces
            s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
            # Break sentence in the token, remove empty tokens
            tokens = [token for token in s.split(" ") if token != ""]
    
            # Use the zip function to help us generate n-grams
            # Concatentate the tokens into ngrams and return
            unigrams = zip(*[tokens[i:] for i in range(1)])
            bigrams = zip(*[tokens[i:] for i in range(2)])
            #ngrams = unigram.append(bigram)
            a = [" ".join(unigram) for unigram in unigrams]
            b = [" ".join(bigram) for bigram in bigrams]
            c = a+b
            #print(a)
            return c
        dataset = []
        df1 = pd.read_csv(Task0().output().path)
        #df = pd.read_csv("Z:/ADS/a/textrank_final.csv")
        with open(Task0().output().path,encoding="utf8", newline='') as myFile:
            reader1 = csv.reader(myFile)
            for row1 in reader1:
                data = row1[2]
        #print(data)
                data = data.lower()
                stop_words = set(stopwords.words("english"))
        #remove tags
                data=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",data)
        # remove special characters and digits
                data=re.sub("(\\d|\\W)+"," ",data)
        ##Convert to list from string
                data = data.split()
        #Lemmatisation
                lem = WordNetLemmatizer()
                data = [lem.lemmatize(word) for word in data if not word in stop_words]
                data = " ".join(data)
                dataset.append(data)
        x = 1
        abc=[]
        df = pd.read_csv(Task1().output().path,names=["0","words"])
        with open(Task1().output().path, newline='') as myFile:
            reader = csv.reader(myFile)
            for row in reader:
                abc.append(row[1])
        mapping = {}
        for text in dataset:
            thisline = generate_ngrams(text, n=2)
            for q in abc:
        
        #print(q)
                for wd in thisline:
            #print(wd)
                    if q == wd:
                        if wd in mapping:
                            mapping[wd] +=1
                 

                    else:
                        mapping.update({wd:1})
                        


            df[x]= df['words'].map(mapping)
    #print(df)
            x +=1
            mapping.clear()
        df2_transposed = df.transpose() # or df2.transpose()
        df2_transposed = df2_transposed.replace(np.nan, 0)
        df2_transposed = df2_transposed.drop("0", axis=0)
        df2_transposed = df2_transposed.drop(columns=0)
        #df2_transposed.to_csv("Z:/ADS/project/pipeline/wordcount_dataset.csv",index=False,header=False, encoding='utf8')
        df2_transposed.to_csv(self.output().path,index=False,header=False, encoding='utf8')
        #print('Task1 Completed')
        
    def output(self):
        return luigi.LocalTarget(os.getcwd()+"/wordcount_dataset.csv")
class Task3(luigi.Task):
    def requires(self):
        yield Task2()
    def run(self):
        #df = pd.read_csv(self.input())
        
        dataset = pd.read_csv(Task2().output().path)
        df = dataset.copy()
        df['Good Cost']=df.iloc[:,0:28].astype(bool).sum(axis=1)
        df['Bad Cost']=df.iloc[:,28:46].astype(bool).sum(axis=1)
        df['Good Recommendation']=df.iloc[:,46:73].astype(bool).sum(axis=1)
        df['Bad Recommendation']=df.iloc[:,73:99].astype(bool).sum(axis=1)
        df['Good Stability']=df.iloc[:,99:121].astype(bool).sum(axis=1)
        df['Bad Stability']=df.iloc[:,121:148].astype(bool).sum(axis=1)
        df['Ads']=df.iloc[:,149:172].astype(bool).sum(axis=1)
        df['No Ads']=df.iloc[:,172:190].astype(bool).sum(axis=1)
        df1 = pd.DataFrame()
        df1 = df[['Good Cost','Bad Cost','Good Recommendation','Bad Recommendation',
                  'Good Stability','Bad Stability','Ads','No Ads']]
        data = pd.read_csv(Task0().output().path,header=0,encoding='utf-8')
        df1 = df1.drop([0],axis = 0)
        df2 = data['App']
        df2.index += 1
        result = pd.concat([df1, df2], axis=1)
        result = result.replace({'Spotify': '1'})
        result = result.replace({'Pandora':'2'})
        result = result.replace({'AppleMusic':'3'})
        result = result.replace({'AmazonMusic':'4'})
        #result.to_csv('z:/ADS/project/pipeline/model_input.csv',index = None)
        result.to_csv(self.output().path,index=None)
        #print('Task2 Completed')
        with open(Task2().output().path, 'rb') as csv:
            dbx.files_upload(csv.read(),'/DataScienceTeam9/wordcount_dataset.csv', mode=dropbox.files.WriteMode("overwrite"))
    def output(self):
        return luigi.LocalTarget(os.getcwd()+"/model_input.csv")
class Task4(luigi.Task):
    def requires(self):
        yield Task3()
    def run(self):
        
        dataset = pd.read_csv(Task0().output().path,header=0,encoding='utf-8')
        df = dataset.copy()
        analyser = SentimentIntensityAnalyzer()
        scores = []
        for items in df['Review']:
            score = analyser.polarity_scores(items)
            scores.append(score)
        final = []
        for items in scores:
            if items['compound'] >= 0.05:
                final.append("Positive")
            elif items['compound'] <= -0.05:
                final.append("Negative")
            else:
                final.append("Neutral")

        final = pd.DataFrame(final,columns = ['Vader Sentiment'])
        df['Vader Sentiment'] = final
        df.to_csv(self.output().path,index=None)
        with open(Task3().output().path, 'rb') as csv:
            dbx.files_upload(csv.read(),'/DataScienceTeam9/model_input.csv', mode=dropbox.files.WriteMode("overwrite"))
    def output(self):
        return luigi.LocalTarget(os.getcwd()+"/Vader_output.csv")
class Task5(luigi.Task):
    def requires(self):
        yield Task4()
        #yield Task2()
    def run(self):
        
        df = pd.read_csv(Task4().output().path)
        df1 = pd.read_csv(Task3().output().path)
        df1 = df1[['Good Cost','Bad Cost','Good Recommendation','Bad Recommendation',
                  'Good Stability','Bad Stability','Ads','No Ads']]
        result = pd.concat([df, df1], axis=1)
        result.to_csv(self.output().path,index=None)
        with open(Task4().output().path, 'rb') as csv:
            dbx.files_upload(csv.read(),'/DataScienceTeam9/Vader_output.csv', mode=dropbox.files.WriteMode("overwrite"))
    def output(self):
        return luigi.LocalTarget(os.getcwd()+"/merged_data.csv")
        
class Task6(luigi.Task):
    def requires(self):
        yield Task5()
        
    def run(self):
       
        features = pd.read_csv(Task3().output().path)
        df = features.copy()
        df.loc[(df['Good Cost']>0),'Good Cost']=1
        df.loc[(df['Good Cost'].isnull()),'Good Cost']=0
        df.loc[(df['Bad Cost']>0),'Bad Cost']=1
        df.loc[(df['Bad Cost'].isnull()),'Bad Cost']=0
        df.loc[(df['Good Recommendation']>0),'Good Recommendation']=1
        df.loc[(df['Good Recommendation'].isnull()),'Good Recommendation']=0
        df.loc[(df['Bad Recommendation']>0),'Bad Recommendation']=1
        df.loc[(df['Bad Recommendation'].isnull()),'Bad Recommendation']=0
        df.loc[(df['Good Stability']>0),'Good Stability']=1
        df.loc[(df['Good Stability'].isnull()),'Good Stability']=0
        df.loc[(df['Bad Stability']>0),'Bad Stability']=1
        df.loc[(df['Bad Stability'].isnull()),'Bad Stability']=0
        df.loc[(df['Ads']>0),'Ads']=1
        df.loc[(df['Ads'].isnull()),'Ads']=0
        df.loc[(df['No Ads']>0),'No Ads']=1
        df.loc[(df['No Ads'].isnull()),'No Ads']=0
        # Labels are the values we want to predict
        labels = np.array(df['App'])
        # Remove the labels from the features
        # axis 1 refers to the columns
        features= df.drop('App', axis = 1)
        # Saving feature names for later use
        feature_list = list(features.columns)
        # Convert to numpy array
        features = np.array(features)
        # Split the data into training and testing sets
        train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)
        model_nb = MultinomialNB().fit(train_features, train_labels)
        test_test = [[0,0,1,0,0,0,0,0]]
        predict_nb_test = model_nb.predict(test_test)
        if predict_nb_test == 1:
            print('Spotify')
        elif predict_nb_test == 2:
            print('Pandora')
        elif predict_nb_test == 3:
            print('AppleMusic')
        elif predict_nb_test == 4:
            print('AmazonMusic')
        with open(Task5().output().path, 'rb') as csv:
            dbx.files_upload(csv.read(),'/DataScienceTeam9/merged_data.csv', mode=dropbox.files.WriteMode("overwrite"))

if __name__ == "__main__":
    luigi.build([Task6()],local_scheduler=True)
    

DEBUG: Checking if Task6() is complete
C:\Users\tawde\Anaconda3\lib\site-packages\luigi\worker.py:401: UserWarning: Task Task6() without outputs has no custom complete() method
  is_complete = task.complete()
DEBUG: Checking if Task5() is complete
INFO: Informed scheduler that task   Task6__99914b932b   has status   PENDING
DEBUG: Checking if Task4() is complete
INFO: Informed scheduler that task   Task5__99914b932b   has status   PENDING
DEBUG: Checking if Task3() is complete
INFO: Informed scheduler that task   Task4__99914b932b   has status   PENDING
DEBUG: Checking if Task2() is complete
INFO: Informed scheduler that task   Task3__99914b932b   has status   PENDING
DEBUG: Checking if Task1() is complete
INFO: Informed scheduler that task   Task2__99914b932b   has status   PENDING
INFO: Informed scheduler that task   Task1__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
DEBUG: Asking scheduler for work...
DEBUG: Pending tasks: 5
INFO: